# Modeling Neighborhood Dynamics

We will use the Columbus MSA as an example to illustrate the `dynamics` module of `geosnap`.

* Modeling transition rates
* Analyzing whole neighborhood sequence

## Data preperation

* Geodemographics for neighborhood segamentation

In [1]:
from geosnap.data import Community, data_store as store
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os
sns.set_context('notebook')
import warnings
warnings.filterwarnings("ignore")

In [2]:
columbusfips=store.msas()[store.msas().name.str.startswith('Columbus, OH')].geoid.values[0]

In [3]:
columbusfipsd

'18140'

In [4]:
columbus = Community.from_ltdb(msa_fips=columbusfips)

In [5]:
columbus.cluster(columns=['median_household_income', 'p_poverty_rate', 'p_edu_college_greater', 'p_unemployment_rate'], method='ward')

In [8]:
columbus.gdf

,n_asian_under_15,n_black_under_15,n_hispanic_under_15,n_native_under_15,n_white_under_15,n_persons_under_18,n_asian_over_60,n_black_over_60,n_hispanic_over_60,n_native_over_60,...,n_total_housing_units_sample,p_nonhisp_white_persons,p_white_over_60,p_black_over_60,p_hispanic_over_60,p_native_over_60,p_asian_over_60,p_disabled,geometry,ward
geoid,,,,,,,,,,,,,,,,,,,,,
39041010100,NaN,73.000000,NaN,NaN,1238.000000,1483.000000,NaN,32.000000,NaN,NaN,...,2001.000000,NaN,14.114495,0.451213,NaN,NaN,NaN,4.244219,"POLYGON ((-83.082292 40.29946, -83.082549 40.3...",3
39041010200,NaN,5.472672,NaN,NaN,1426.479469,1599.197822,NaN,5.387974,NaN,NaN,...,1529.132207,NaN,12.308566,0.120177,NaN,NaN,NaN,8.510786,"POLYGON ((-83.071978 40.327509, -83.02909 40.3...",1
39041010420,NaN,1.505399,NaN,NaN,126.788033,147.696350,NaN,0.167267,NaN,NaN,...,126.286232,NaN,9.413255,0.042212,NaN,NaN,NaN,3.292529,"POLYGON ((-83.09434499999999 40.328399, -83.09...",0
39041010421,NaN,1.095593,NaN,NaN,92.273573,107.490183,NaN,0.121733,NaN,NaN,...,91.908363,NaN,9.413270,0.042212,NaN,NaN,NaN,3.292555,"POLYGON ((-83.126183 40.306402, -83.1249409999...",0
39041010422,NaN,2.941651,NaN,NaN,247.761943,288.619354,NaN,0.326855,NaN,NaN,...,246.780934,NaN,9.413450,0.042211,NaN,NaN,NaN,3.292857,"POLYGON ((-83.12404599999999 40.330013, -83.11...",0
39041010520,NaN,63.361351,NaN,NaN,340.952014,447.127047,NaN,14.887277,NaN,NaN,...,394.780062,NaN,9.258405,1.201641,NaN,NaN,NaN,6.239844,"POLYGON ((-83.128518 40.278896, -83.128158 40....",1
39041010530,NaN,14.501827,NaN,NaN,78.019829,102.310387,NaN,3.407929,NaN,NaN,...,90.346382,NaN,9.258312,1.202046,NaN,NaN,NaN,6.240409,"POLYGON ((-83.08266499999999 40.292869, -83.07...",1
39041011101,NaN,NaN,NaN,NaN,417.840286,473.756582,NaN,NaN,NaN,NaN,...,414.010388,NaN,15.347432,NaN,NaN,NaN,NaN,7.854984,POLYGON ((-83.03037999999999 40.30671299999999...,1
39041011102,NaN,NaN,NaN,NaN,672.996167,763.058005,NaN,NaN,NaN,NaN,...,666.827523,NaN,15.347431,NaN,NaN,NaN,NaN,7.854984,"POLYGON ((-83.030997 40.296993, -83.030845 40....",1


In [7]:
columbus.gdf.describe()

,n_asian_under_15,n_black_under_15,n_hispanic_under_15,n_native_under_15,n_white_under_15,n_persons_under_18,n_asian_over_60,n_black_over_60,n_hispanic_over_60,n_native_over_60,...,n_white_persons,year,n_total_housing_units_sample,p_nonhisp_white_persons,p_white_over_60,p_black_over_60,p_hispanic_over_60,p_native_over_60,p_asian_over_60,p_disabled
count,1672.000000,1940.000000,1672.000000,1672.000000,2008.000000,2008.000000,1241.000000,1509.000000,1241.000000,1241.000000,...,336.000000,2008.000000,2008.000000,1672.000000,1577.000000,1509.000000,1241.000000,1241.000000,1241.000000,1577.000000
mean,15.714495,125.944063,21.164742,1.227273,615.756615,940.151154,2.551090,39.564625,1.840247,0.473734,...,2398.874036,1991.235060,1427.053434,81.554311,10.855994,1.183477,0.052036,0.013938,0.068217,7.961886
std,43.295055,262.036518,48.194609,3.928431,481.960644,588.595403,4.580426,114.205586,2.772873,0.967512,...,1840.341717,13.841891,765.216684,24.173496,7.109066,3.332036,0.081724,0.030484,0.113736,4.503483
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.031740,1970.000000,0.009182,1.909059,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.198226,0.041828,0.000000,205.000000,513.750000,0.000000,0.000402,0.000000,0.000000,...,734.794701,1980.000000,942.982471,78.567439,5.383832,0.000028,0.000000,0.000000,0.000000,4.634939
50%,2.842409,16.000000,7.518665,0.000000,549.654396,878.010657,0.641971,3.028295,0.994432,0.000000,...,2158.864707,1990.000000,1381.666199,92.075976,10.420235,0.111152,0.025954,0.000000,0.018832,6.996625
75%,14.005660,107.714780,20.000000,1.001310,908.500000,1284.884796,3.000000,19.337119,3.000000,0.811625,...,3698.469995,2000.000000,1864.957245,97.130156,15.100684,0.575374,0.072308,0.018882,0.093480,10.460624
max,1139.000000,2455.005536,686.000000,93.000000,3260.901611,4810.000000,36.000000,1327.819259,27.000000,10.000000,...,10792.342843,2010.000000,7848.000000,99.795437,51.639799,29.071244,1.088271,0.339098,0.958160,31.075419
